In [1]:
import ipyvuetify as v
from sepal_ui import sepalwidgets as sw

HTML(value='\n<style>\n.leaflet-pane {\n    z-index : 2 !important;\n}\n.leaflet-top, .leaflet-bottom {\n    z…

In [2]:
class less_more(v.Switch):
    
    def __init__(self, **kwarg):
        
        self.items = ['less than', 'more than']
        
        super().__init__(
            v_model = None,
            label = self.items[0]
        )
        
        def on_change(change):
            self.label = self.items[change['new']]
        
        self.observe(on_change, 'v_model')
        
    

In [3]:
class Criteria (v.Row):
    
    def __init__(self, name = 'name', value = 0, **kwarg):
        
        self.name = name
        
        super().__init__(
            children = [
                v.Flex(class_ = 'align-center', xs2 = True, children = [v.Html(class_ = 'align-center', tag = 'h4', children = [name])]),
                v.Flex(xs2 = True, children = [less_more()]),
                v.Flex(xs1 = True, children = [v.TextField(type="number", v_model = None)])
            ],
            **kwarg
        )
        
        self.class_ = self.class_ + ' pa-5'

In [4]:
from sepal_ui.scripts import utils as su

class step2(sw.Tile):
    
    def __init__(self, **kwargs):
        
        title = "Constraints" 
        id_= 'nested_widget'
        
#         tree_cover_switch = v.Switch(
#             v_model = False,
#             label = "Are percent tree cover maps avalaible for biophysical potential?"
#         )
        
        txt = """ 
        Select constraints from the dropdown menu below to add a limitation to consider.  
        For each constraint, choose values greater than or less than a particular value.  
        Note that some constraint are scaled differntly so please read their short description after selection to assign appropriate values.
        """
        
        tile_txt = sw.Markdown(txt)
        
        criterias = [
            'Bare land',
            'Shrub land',
            'Agricultural land',
            'Annual rainfall',
            'Population',
            'Elevation',
            'Slope',
            'Tree cover',
            'Protected area',
            'Opportunity cost'
        ]
        
        self.critera_select = v.Select(
            v_model = None,
            items = criterias,
            label = 'Criteria',
            multiple = True
        )
        
        self.criterias_values = [Criteria(name=name, class_='d-none') for name in criterias]
        
        super().__init__(
            id_, 
            title, 
            inputs = [
#                 tree_cover_switch, 
                tile_txt, 
                self.critera_select
            ] + self.criterias_values, 
            **kwargs
        )
        self.children[0].elevation = 0
        
        self.hide_criterias()

    def hide_criterias(self):
        
        def on_change(change):
            
            for criteria in self.criterias_values:
                if criteria.name in change['new']:
                    su.show_component(criteria)
                else:
                    su.hide_component(criteria)
        
        self.critera_select.observe(on_change, 'v_model')
        
        

In [5]:
class step3(sw.Tile):
    
    def __init__(self, **kwargs):
        
        title = "Restoration potential and land use" 
        id_ = 'nested_widget'
        
        txt = """Max % tree cover in land use classes  
        """
        tile_txt = sw.Markdown(txt)
        

        land_use = v.Select(
            label = "Which land uses allow restoration?",
            items = [
                "Agriculture",
                "Protected Areas",
                "Rangeland",
                "Grassland"

            ],
            multiple = True

        )
        
        pcnt_treecover = v.Slider(label=f'Maximum allowable percent tree cover in {land_use.items}' )


    
        super().__init__(
            id_, 
            title, 
            inputs = [
                tile_txt, 
                land_use,
                pcnt_treecover
                
            ],
            **kwargs
        )
        self.children[0].elevation = 0
    

In [6]:
class step4(sw.Tile):
    
    def __init__(self, **kwargs):
        
        title = "Restoration goals"
        id_ = "nested_widget"
        
        goals = [
            'Enhancement of existing areas',
            'Increase the forest cover',
            'Reflect relevant national regulations',
            'Achievement of international commitments',
            'Improve connectivity - landscape biodiversity'
        ]
        
        select = v.Select(
            label = "restoration goals",
            items = goals,
            v_model = None
        )
        
        super().__init__(
            id_, 
            title, 
            inputs = [select],
            **kwargs
        )
        self.children[0].elevation = 0
        
        self.observe(self.on_change, select.v_model)
    def on_change(self, change):
        self.v_model = self.goals['new']

        

In [7]:
class satisfaction_slider(v.Slider):
    
    _labels = [
        'no importance',
        'low importance',
        'neutral',
        'important',
        'very important',
    ]
        
    _colors = [
        'red',
        'orange',
        'yellow accent-3',
        'light-green',
        'green'
    ]
    
    def __init__(self, label, **kwargs):
        
        super().__init__(
            label = label,
            max = 4,
            min = 0,
            #step = 1,
            #ticks = "always",
            #tick_size = 4,
            track_color = 'grey',
            thumb_label = self._labels[4],
            color = self._colors[4],
            v_model = 4
        )
        
        self.observe(self.on_change, 'v_model')
        
    def on_change(self, change):
        self.color = self._colors[change['new']]
        self.thumb_label = self._labels[change['new']]

In [8]:
class step5 (sw.Tile):
    
    def __init__(self, **kwargs):
        
        title = "Restoration priorities"
        id_ = "nested_widget"
        
        super().__init__(
            id_, 
            title, 
            inputs = [
                satisfaction_slider('Production Forest'),
                satisfaction_slider(' Protected Area'),
                satisfaction_slider('Community Foresty'),
                satisfaction_slider('Community Protected Area'),
                satisfaction_slider('Biodiversity conservation corridor'),
                satisfaction_slider('Other private land or public land')
            ],
            **kwargs
        )
        self.children[0].elevation = 0

In [9]:
step2_content = v.StepperContent(
    key=1,
    step=1, 
    children=[step2()]
)  

step3_content = v.StepperContent(
    key=2,
    step=2, 
    children=[step3()]
)

step4_content = v.StepperContent(
    key=3,
    step=3, 
    children=[step4()]
)

step5_content = v.StepperContent(
    key=4,
    step=4, 
    children=[step5()]
)

stepper_children = [
    v.StepperStep(
        key=1,
        complete=False,
        step=1,
        editable=True,
        children=["Constraints"]
    ),
    v.Divider(),
    v.StepperStep(
        key=2,
        complete=False,
        step=2,
        editable=True,
        children=["Restoration potential and land use"]
    ),
    v.Divider(),
    v.StepperStep(
        key=3,
        complete=False,
        step=3,
        editable=True,
        children=["Restoration goals"]
    ),
    v.Divider(),
    v.StepperStep(
        key=4,
        complete=False,
        step=4,
        editable=True,
        children=["Restoration priorities"]
    ),
]

stepper_header = v.StepperHeader(
    children=stepper_children
)

aoi_content = v.Stepper(
        class_="mt-2",
         children=[
             stepper_header,
             v.StepperItems(
                 children=[
                     step2_content,
                     step3_content,
                     step4_content,
                     step5_content
                 ]
             )
         ]
)

In [10]:
title = "Questionnaire Identifying Restoration Objective & Goals"
id_ = "questionnaire_widget"
process_tile = sw.Tile(id_, title, inputs=[aoi_content] )
process_tile

Tile(align_center=True, children=[Card(children=[Html(children=['Questionnaire Identifying Restoration Objecti…

In [13]:
from fra.functions import gee_compute
class compute_params:
    def __init__(self):
        self.aoi_content = None
    def get_params(self):
        aoi_content = self.aoi_content
        self.current_params = gee_compute(aoi_content).prep_run_comp()
    
        return self.current_params

In [14]:

compute_io = compute_params()
setattr(compute_io,'aoi_content', aoi_content)
# compute_io.get_params()


([6.0, 6.0, 6.0, 6.0, 6.0, 0.0, 0.0, 0.0],
 [6.0, 6.0, 6.0, 6.0, 6.0, 0.0, 0.0, 0.0],
 [{'name': 'Shrub land', 'image': True, 'mode': None, 'value': '2'},
  {'name': 'Agricultural land', 'image': True, 'mode': None, 'value': None}],
 [{'label': 'Production Forest', 'value': 4},
  {'label': ' Protected Area', 'value': 4},
  {'label': 'Community Foresty', 'value': 4},
  {'label': 'Community Protected Area', 'value': 4},
  {'label': 'Biodiversity conservation corridor', 'value': 4},
  {'label': 'Other private land or public land', 'value': 4}])